In [ ]:
!pip install ktrain
!pip install transformers

In [ ]:
import ktrain
import random 
import pickle
import torch
import time
import transformers
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras.utils as ku
import torch.nn as nn

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from ktrain import text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# from google.colab import drive

In [ ]:
device = torch.device("cuda")

In [ ]:
%matplotlib inline

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pwd


/content


In [ ]:
cd /content/drive/MyDrive/assignmnet data

/content/drive/MyDrive/assignmnet data


In [ ]:
pwd

'/content/drive/MyDrive/assignmnet data'

In [ ]:
train_data = pd.read_csv("/content/hihi.csv")

In [ ]:
train_data.head()

# Remove column name 'A' 
# haha = train_data.drop(['0'], axis = 1)
# hihi = haha.drop(['0.1'], axis = 1)

In [ ]:
train_data['Labels_num'] = train_data['Labels'].map({'PERSON': 1, 'TIME': 0, 'LOCATION': 2, 'NUMBER': 3, 
'ORGANIZATION': 4, 'DATE': 5, 'DESIGNATION': 6, 'NaN': 7})
# tweets['label_num'] = tweets['label'].map({'h': 1, 'n': 0, 'o': 2})

In [ ]:
# NER_Data = train_data.drop(['Labels'], axis = 1)

In [ ]:
hihi.to_csv("hihi.csv")

In [ ]:
!git clone https://github.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation.git

Cloning into 'SemEval2020-Task4-Commonsense-Validation-and-Explanation'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 36), reused 64 (delta 19), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [ ]:

trf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/train.csv'
train_data = pd.read_csv(trf)
tf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/test.csv'
test_data = pd.read_csv(tf)
vf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/dev.csv'
valid_data = pd.read_csv(vf)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
valid_data

In [ ]:
corr_statement= []
incorr_statement=[]
right_reason1= []
right_reason2 = []

for i in range(len(train_data['Correct Statement'])):
  correct_statement.append([train_data.iloc[i,0], 1])
  incorrect_statement.append([train_data.iloc[i,1], 0])
  right_reason1.append([train_data.iloc[i,2], 1])
  right_reason2.append([train_data.iloc[i,3], 1])

for i in range(len(test_data['Correct Statement'])):
  correct_statement.append([test_data.iloc[i,0], 1])
  incorr_statement.append([test_data.iloc[i,1], 0])
  right_reason1.append([test_data.iloc[i,2], 1])
  right_reason2.append([test_data.iloc[i,3], 1])

for i in range(len(valid_data['Correct Statement'])):
  corr_statement.append([valid_data.iloc[i,0], 1])
  incorrect_statement.append([valid_data.iloc[i,1], 0])
  right_reason1.append([valid_data.iloc[i,2], 1])
  right_reason2.append([valid_data.iloc[i,3], 1])


In [ ]:
data_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(data_list) 
data_list

In [ ]:
test_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(test_list) 
test_list

In [ ]:
validation_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(validation_list) 
validation_list

In [ ]:
column_labels = ['Sentence', 'Labels']
train_data_df = pd.DataFrame(data_list, columns = column_labels)
test_data_df = pd.DataFrame(test_list, columns = column_labels)
dev_data_df = pd.DataFrame(validation_list, columns = column_labels)

In [ ]:
train_data_df.head()

,Sentence,Labels
0,stone isn't a kind of food,1
1,Eating apples is good,1
2,Developed countries are relatively advanced,1
3,The sand found in the sahara desert can cure c...,1
4,"Fish can't fly, they swim.",1


In [ ]:
print(train_data.shape)
train_data.head()

(6277, 3)


,Entities,Labels,Labels_num
0,\nحفیظ,PERSON,1.0
12,\nروز,TIME,0.0
36,\nشارجہ,LOCATION,2.0
40,\nدن,TIME,0.0
49,\nمحمد,PERSON,1.0


In [ ]:
train_data = train_data.dropna(subset=['Labels_num'])

In [ ]:
# column_labels = ['محمد', 'PERSON']
# train_data_df = pd.DataFrame(train_data, columns = column_labels)

(X_train, Y_train) , (X_test, Y_test), preprocess = text.texts_from_df(train_df= train_data, 
                                                                       text_column= 'Entities', 
                                                                       label_columns= 'Labels_num',
                                                                       maxlen = 75)
                                                                      #  preprocess_mode = 'bert'    

/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:605: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')


language: ur
Word Counts: 1036
Nrows: 5649
5649 train sequences
train sequence lengths:
	mean : 1
	95percentile : 1
	99percentile : 1
x_train shape: (5649,75)
y_train shape: (5649, 7)
Is Multi-Label? False
628 test sequences
test sequence lengths:
	mean : 1
	95percentile : 1
	99percentile : 1
x_test shape: (628,75)
y_test shape: (628, 7)


In [ ]:
X_train[0].shape

(26000, 400)

In [ ]:
model = text.text_classifier(name='fasttext', 
                             train_data = (X_train, Y_train),
                             preproc = preprocess)

Is Multi-Label? False
compiling word ID features...
maxlen is 75
done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, Y_train),
                             val_data = (X_test, Y_test),
                             batch_size = 50)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=500)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/500
113/113 [==============================] - 4s 17ms/step - loss: 2.8066 - accuracy: 0.1224 - val_loss: 1.9609 - val_accuracy: 0.1354
Epoch 2/500
113/113 [==============================] - 2s 14ms/step - loss: 2.8137 - accuracy: 0.1081 - val_loss: 1.9734 - val_accuracy: 0.1274
Epoch 3/500
113/113 [==============================] - 2s 14ms/step - loss: 2.7795 - accuracy: 0.1150 - val_loss: 1.9964 - val_accuracy: 0.1274
Epoch 4/500
113/113 [==============================] - 2s 14ms/step - loss: 2.7790 - accuracy: 0.1144 - val_loss: 2.0384 - val_accuracy: 0.1274
Epoch 5/500
113/113 [==============================] - 2s 13ms/step - loss: 2.7632 - accuracy: 0.1081 - val_loss: 2.1084 - val_accuracy: 0.1274
Epoch 6/500
113/113 [==============================] - 2s 14ms/step - loss: 2.7145 - accuracy: 0.1180 - val_loss: 2.1867 - val_accuracy: 0.1258
Epoch 7/500
113/113 [==============================] - 2s 13ms/step - los

In [ ]:
learner.view_top_losses(n=500,preproc=preprocess)

----------
id:38 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:108 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:128 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:179 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:209 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:399 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:408 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:417 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:432 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:448 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:567 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:583 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id:618 | loss:5.7 | true:Labels_num_3.0 | pred:Labels_num_1.0)


----------
id

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
predictor.save('/content/drive/My Drive/bert')

In [ ]:
result = predictor.predict(list(test_data_df['Sentence']))
result

['Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',

In [ ]:
for i in range(len(result)):
  if result[i] == 'Labels':
    result[i] = 1
  else:
    result[i] = 0

In [ ]:
result

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
test_data_df

,Sentence,Labels
0,Chairs have four legs,1
1,The light of the moon is the result of reflect...,1
2,Cat food and human food contain similar nutrients,1
3,human can't live without water,1
4,The monkey is much larger than a cup.,1
...,...,...
25995,Crying can show people happy and happy,1
25996,Apple is a technology company that sells perso...,1
25997,The snake sleeps on the mice that come through...,0
25998,bike has two tires and square has four sides,1


In [ ]:
count=0
for i in range (len(result)):
  if result[i] == test_data_df.iloc[i,1]:
    count+=1
accuracy_test = (count/len(result)) * 100
print("Accuracy on Test Dataset: ", accuracy_test, "%")

NameError: ignored

In [ ]:
predictor_load = ktrain.load_predictor('/content/drive/MyDrive/bert')

In [ ]:
t_f = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/Test Data/subtaskA_test_data.csv'
test_data = pd.read_csv(t_f)

In [ ]:
result = predictor.predict(list(test_data['sent1']))
for i in range(len(result)):
  if result[i] == 'Labels':
    result[i] = 1
  else:
    result[i] = 0
result 

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
len(result)

1000

In [ ]:
t_ids = list(test_data['id'])
len(t_ids)

1000

In [ ]:
t_list = []
for i in range(len(result)):
  t_list.append([t_ids[i],result[i]])
t_list

[[1175, 1],
 [452, 1],
 [275, 1],
 [869, 1],
 [50, 1],
 [1155, 1],
 [1395, 1],
 [967, 1],
 [906, 1],
 [1465, 1],
 [1680, 1],
 [1964, 1],
 [830, 1],
 [174, 1],
 [461, 1],
 [1978, 1],
 [495, 1],
 [1388, 1],
 [1572, 1],
 [305, 1],
 [1695, 1],
 [1886, 1],
 [878, 1],
 [754, 1],
 [1444, 1],
 [1065, 1],
 [543, 1],
 [518, 1],
 [692, 1],
 [1296, 1],
 [1893, 1],
 [1915, 1],
 [418, 1],
 [953, 1],
 [1148, 1],
 [1943, 1],
 [105, 1],
 [1222, 1],
 [400, 1],
 [1610, 1],
 [230, 1],
 [1172, 1],
 [947, 1],
 [1144, 1],
 [1017, 1],
 [1063, 1],
 [930, 1],
 [83, 1],
 [1145, 1],
 [363, 1],
 [338, 1],
 [391, 1],
 [1139, 1],
 [688, 1],
 [704, 1],
 [1544, 1],
 [1760, 1],
 [303, 1],
 [850, 1],
 [924, 1],
 [1840, 1],
 [292, 1],
 [7, 1],
 [800, 1],
 [1177, 1],
 [259, 1],
 [274, 1],
 [1031, 1],
 [377, 1],
 [1225, 1],
 [396, 1],
 [1860, 1],
 [1500, 1],
 [1625, 1],
 [896, 1],
 [1704, 1],
 [1280, 1],
 [125, 1],
 [1509, 1],
 [458, 1],
 [697, 1],
 [1483, 1],
 [317, 1],
 [757, 1],
 [770, 1],
 [1976, 1],
 [970, 1],
 [1712,

In [ ]:
test_data = pd.DataFrame(t_list)

In [ ]:
gfg_csv_data = test_data.to_csv('subtaskA_answers.csv', index = True) 